-----
-----
-----


**Batch Renorm şu: BatchNorm’un “küçük batch yüzünden saçmalamasını” azaltmak için çıkan bir yamadır.**


## 1) Sorun: BatchNorm küçük batch’te niye bozuluyor?

BN eğitimde şunu yapıyor:

* Batch’ten ortalama (μ_B) ve varyans (σ_B²) hesaplıyor

* Sonra her şeyi ona göre normalize ediyor

* Ama batch küçükse (mesela B=2):

* μ_B ve σ_B çok gürültülü olur

* Her iterasyonda başka başka değerler çıkar

Modelin içindeki aktivasyon dağılımı zıplaya zıplaya gider
**→ eğitim kararsız, performans düşer**

Ayrıca:

* Train’de batch stats kullanıyorsun

* Eval’da running stats (μ_R, σ_R) kullanıyorsun
**→ train–eval farkı büyür**

>### Batch Renorm ne yapıyor?

Batch Renorm diyor ki:

**“Tamam batch istatistiğini kullan ama running istatistiğe göre düzelt.**

* Batch çok sapıtırsa da kırbaçla, sınırlı tut (clamp).”

Yani eğitimde 2 kaynak var:

* Batch stats: μ_B, σ_B (anlık, gürültülü)

* Running stats: μ_R, σ_R (birikmiş, daha stabil)

Batch Renorm bu ikisini birleştiriyor.


>### r ve d dediğimiz şey ne?

Batch Renorm iki düzeltme katsayısı hesaplıyor:

* r: “ölçek düzeltmesi”

* d: “kaydırma düzeltmesi”

Sezgi:

* Batch’in std’si running’den fazla farklıysa → r ile düzelt

* Batch’in mean’i running’den fazla farklıysa → d ile düzelt

Ve bunları clamp ediyor:

* r çok uçmasın

* d çok uçmasın

**Batch Renorm = BatchNorm + (running istatistiklere göre ölçek/kaydırma düzeltmesi) + clamp**

----
----
----


# Batch Renormalization (Batch Renorm) — Baştan Sona (Tek GPU / Küçük Batch Odaklı)

Bu notebook, **Batch Renormalization** konusunu **temelden ileri seviyeye** kadar anlatır ve PyTorch’ta kullanılabilir bir implementasyon verir.

- BatchNorm’un küçük batch’te neden zorlandığı  
- Batch Renorm’un fikri (**r**, **d** düzeltmeleri)  
- Train/Eval uyumsuzluğunu nasıl azalttığı  
- Hangi senaryolarda mantıklı / mantıksız  
- PyTorch’ta **BatchRenorm2d** implementasyonu (drop‑in replacement)  
- Pratik checklist ve tipik hatalar  
- Mini demo: küçük batch’te BN vs BatchRenorm sezgisi  

> Not: Bu dosya tek GPU / tek makine kullanımına uygundur. SyncBN gibi “multi‑GPU şart” değildir.



## 1) BatchNorm’u hatırlayalım (problem nerede başlıyor?)

Conv için **BatchNorm2d** genelde kanallar üzerinde çalışır ve istatistikleri **(B, H, W)** eksenlerinde toplar:

- Batch mean:  \(\mu_B\)  
- Batch var:   \(\sigma_B^2\)

Normalize:
\[
\hat{x} = \frac{x - \mu_B}{\sqrt{\sigma_B^2 + \epsilon}}
\]
Affine:
\[
y = \gamma \hat{x} + \beta
\]

### Train vs Eval farkı
- **Train:** \(\mu_B, \sigma_B\) mini‑batch’ten gelir.
- **Eval:** “running” istatistikler kullanılır: \(\mu_R, \sigma_R\)

### BN’in temel zayıflığı
Per‑batch istatistikleri güvenilir değilse (batch küçükse):
- \(\mu_B, \sigma_B\) gürültülü olur
- Train sırasında normalize edilen dağılım “zıplar”
- Eval’da running istatistikle farklı normalize edilir ⇒ **train‑eval mismatch** artar



## 2) Batch Renormalization nedir? (ana fikir)

Batch Renorm (Ioffe, 2017) şunu hedefler:

> “Train sırasında batch istatistikleri ile running istatistikleri arasında **kontrollü bir köprü** kur.”

Yani train sırasında tamamen \(\mu_B, \sigma_B\) ile gitmek yerine,
running (\(\mu_R, \sigma_R\)) istatistiklerini referans alır ve batch’e bir düzeltme uygular.

### 2.1. r ve d düzeltmeleri
\[
r = \frac{\sigma_B}{\sigma_R}
\]
\[
d = \frac{\mu_B - \mu_R}{\sigma_R}
\]

Sonra normalize edilmiş aktivasyonu şu şekilde düzeltir:
\[
\hat{x}_{BR} = r \cdot \hat{x}_{BN} + d
\]

Burada:
- \(\hat{x}_{BN}\) = normal BN ile normalize edilmiş değer
- r, d = batch istatistiğini running istatistiğe “yaklaştıran” düzeltmeler

### 2.2. Neden clamp var?
r ve d sınırsız olursa küçük batch’te düzeltme patlayabilir. Bu yüzden:

\[
r \in [1/r_{max}, r_{max}], \quad d \in [-d_{max}, d_{max}]
\]

rmax ve dmax genelde **eğitim boyunca yavaş yavaş artırılır** (schedule).
Başta sistem daha “BN gibi” davranır, zamanla renorm etkisi artar.



## 3) Batch Renorm neyi düzeltir?

### 3.1. Küçük batch kararsızlığı
Batch küçükken BN’in istatistiği gürültülüdür.
Batch Renorm, gürültüyü tamamen yok etmez ama running istatistikleri referans aldığı için
normalize edilen dağılımı daha “tutarlı” hale getirir.

### 3.2. Train‑Eval mismatch
BN’de train normalize = batch stats, eval normalize = running stats.
Küçük batch’te bu fark büyür. Batch Renorm, train sırasında da running’e bağlı bir düzeltme kullandığı için
train ve eval davranışını birbirine yaklaştırır.

### 3.3. “BN’yi çöpe atma” değil
Batch Renorm hâlâ batch istatistiğini kullanır ama onu kontrollü biçimde running’e bağlar.



## 4) Ne zaman kullanmalı? (karar çerçevesi)

### Kullan ✅
- Tek GPU / küçük batch (1–8 gibi)
- CNN tabanlı model (Conv ağırlıklı)
- BN’den vazgeçmek istemiyorsun ama GN’ye geçmek de istemiyorsun
- Finetune’da BN kararsızlığı yaşıyorsun

### Kullanma / dikkatli ol ⚠️
- Batch zaten büyükse: çoğu zaman gereksiz
- Çok kısa eğitim: rmax/dmax schedule’ı oturmadan eğitim biter

### Alternatifler
- **GroupNorm:** batch bağımsız, çok stabil
- **BN freeze:** pretrained backbone finetune’da sık kullanılır



## 5) Batch Renorm parametreleri (pratik)

Tipik parametreler:
- `eps`: sayısal stabilite
- `momentum`: running stats güncellemesi
- `rmax`, `dmax`: clamp limitleri
- `warmup_steps` veya schedule: rmax/dmax zamanla artar

Örnek schedule fikri:
- Başlangıç: rmax=1.0, dmax=0.0  (tam BN gibi)
- Zamanla: rmax → 3.0, dmax → 5.0 gibi



## 6) PyTorch’ta BatchRenorm2d implementasyonu

PyTorch’un core `nn` içinde BatchRenorm yok. O yüzden “drop‑in replacement” bir modül yazacağız.

Hedef:
- `nn.BatchNorm2d` yerine kullanılabilsin
- `train()` modunda batch + running stats üzerinden r,d hesaplasın
- `eval()` modunda normal BN gibi running stats kullansın
- Running mean/var’ı momentum ile güncellesin


In [1]:
import torch
import torch.nn as nn

class BatchRenorm2d(nn.Module):
    def __init__(
        self,
        num_features: int,
        eps: float = 1e-5,
        momentum: float = 0.1,
        affine: bool = True,
        track_running_stats: bool = True,
        rmax: float = 3.0,
        dmax: float = 5.0,
        warmup_steps: int = 5000,
    ):
        super().__init__()
        self.num_features = num_features
        self.eps = eps
        self.momentum = momentum
        self.affine = affine
        self.track_running_stats = track_running_stats

        self.rmax_target = float(rmax)
        self.dmax_target = float(dmax)
        self.warmup_steps = int(warmup_steps)

        if self.affine:
            self.weight = nn.Parameter(torch.ones(num_features))
            self.bias = nn.Parameter(torch.zeros(num_features))
        else:
            self.register_parameter('weight', None)
            self.register_parameter('bias', None)

        if self.track_running_stats:
            self.register_buffer('running_mean', torch.zeros(num_features))
            self.register_buffer('running_var', torch.ones(num_features))
            self.register_buffer('num_batches_tracked', torch.tensor(0, dtype=torch.long))
        else:
            self.register_buffer('running_mean', None)
            self.register_buffer('running_var', None)
            self.register_buffer('num_batches_tracked', None)

    def _current_clamp(self):
        if not self.track_running_stats:
            return self.rmax_target, self.dmax_target

        t = int(self.num_batches_tracked.item())
        alpha = 1.0 if self.warmup_steps <= 0 else min(1.0, t / self.warmup_steps)

        rmax = 1.0 + alpha * (self.rmax_target - 1.0)
        dmax = 0.0 + alpha * (self.dmax_target - 0.0)
        return rmax, dmax

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        if x.dim() != 4:
            raise ValueError('BatchRenorm2d expects input of shape (N, C, H, W)')

        if self.track_running_stats:
            self.num_batches_tracked += 1

        if self.training:
            mean = x.mean(dim=(0, 2, 3))
            var = x.var(dim=(0, 2, 3), unbiased=False)
            std = torch.sqrt(var + self.eps)

            if self.track_running_stats:
                r_std = torch.sqrt(self.running_var + self.eps)

                r = (std / r_std).detach()
                d = ((mean - self.running_mean) / r_std).detach()

                rmax, dmax = self._current_clamp()
                r = torch.clamp(r, 1.0 / rmax, rmax)
                d = torch.clamp(d, -dmax, dmax)
            else:
                r = 1.0
                d = 0.0

            x_hat = (x - mean[None, :, None, None]) / std[None, :, None, None]

            if self.track_running_stats:
                x_hat = x_hat * r[None, :, None, None] + d[None, :, None, None]

                with torch.no_grad():
                    self.running_mean = (1 - self.momentum) * self.running_mean + self.momentum * mean
                    self.running_var  = (1 - self.momentum) * self.running_var  + self.momentum * var

        else:
            if not self.track_running_stats:
                raise RuntimeError('Eval mode requires track_running_stats=True')
            x_hat = (x - self.running_mean[None, :, None, None]) / torch.sqrt(
                self.running_var[None, :, None, None] + self.eps
            )

        if self.affine:
            x_hat = x_hat * self.weight[None, :, None, None] + self.bias[None, :, None, None]

        return x_hat

m = BatchRenorm2d(8)
x = torch.randn(4, 8, 16, 16)
y = m(x)
y.shape

torch.Size([4, 8, 16, 16])


### 6.1. r ve d neden `detach()`?
r ve d’yi `detach()` yapmanın amacı:
- Renorm düzeltmelerinin gradyan yolunu “garipleştirmesini” istememek
- Daha stabil optimizasyon

Pratikte bu genelde daha güvenli davranır.



## 7) BN katmanlarını BatchRenorm’a dönüştürme (model içinde)

Elinde BN kullanan bir CNN varsa, modülü dolaşıp `BatchNorm2d` → `BatchRenorm2d` çevirebilirsin.


In [ ]:
import torch.nn as nn
import torch

def convert_bn_to_batchrenorm(module: nn.Module, rmax=3.0, dmax=5.0, warmup_steps=5000):
    for name, child in module.named_children():
        if isinstance(child, nn.BatchNorm2d):
            brn = BatchRenorm2d(
                num_features=child.num_features,
                eps=child.eps,
                momentum=child.momentum if child.momentum is not None else 0.1,
                affine=child.affine,
                track_running_stats=child.track_running_stats,
                rmax=rmax,
                dmax=dmax,
                warmup_steps=warmup_steps,
            )

            if child.affine:
                with torch.no_grad():
                    brn.weight.copy_(child.weight)
                    brn.bias.copy_(child.bias)

            if child.track_running_stats:
                with torch.no_grad():
                    brn.running_mean.copy_(child.running_mean)
                    brn.running_var.copy_(child.running_var)
                    brn.num_batches_tracked.copy_(child.num_batches_tracked)

            setattr(module, name, brn)
        else:
            convert_bn_to_batchrenorm(child, rmax=rmax, dmax=dmax, warmup_steps=warmup_steps)
    return module

class MiniCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(3, 16, 3, padding=1, bias=False),
            nn.BatchNorm2d(16),
            nn.ReLU(inplace=True),
            nn.Conv2d(16, 32, 3, padding=1, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
        )
    def forward(self, x): return self.net(x)

model = MiniCNN()
print("Önce:\n", model)
model = convert_bn_to_batchrenorm(model, rmax=3.0, dmax=5.0, warmup_steps=2000)
print("\nSonra:\n", model)

Önce:
 MiniCNN(
  (net): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
  )
)

Sonra:
 MiniCNN(
  (net): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchRenorm2d()
    (2): ReLU(inplace=True)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (4): BatchRenorm2d()
    (5): ReLU(inplace=True)
  )
)



## 8) Mini demo: Küçük batch’te BN vs BatchRenorm sezgisi

Bu demo tam eğitim değil; **küçük batch istatistiğinin oynaklığını** ve
Batch Renorm’un running’e bağlanmasının etkisini sezdirir.


In [3]:
import torch
import torch.nn as nn

torch.manual_seed(0)

x_small = torch.randn(2, 16, 32, 32)    # küçük batch
x_big   = torch.randn(32, 16, 32, 32)   # büyük batch

bn = nn.BatchNorm2d(16).train()
brn = BatchRenorm2d(16, rmax=3.0, dmax=5.0, warmup_steps=0).train()

for _ in range(20):
    _ = brn(torch.randn(8, 16, 32, 32))

y_bn_small = bn(x_small)
y_bn_big   = bn(x_big)

y_brn_small = brn(x_small)
y_brn_big   = brn(x_big)

def summarize(t):
    return {
        'mean': float(t.mean()),
        'std': float(t.std(unbiased=False)),
        'min': float(t.min()),
        'max': float(t.max()),
    }

print('BN  küçük batch:', summarize(y_bn_small))
print('BN  büyük batch:', summarize(y_bn_big))
print('BRN küçük batch:', summarize(y_brn_small))
print('BRN büyük batch:', summarize(y_brn_big))


BN  küçük batch: {'mean': 1.1059455573558807e-09, 'std': 0.9999949932098389, 'min': -4.318045139312744, 'max': 4.028398036956787}
BN  büyük batch: {'mean': 1.8044374883174896e-09, 'std': 0.9999949932098389, 'min': -4.584938049316406, 'max': 4.676060676574707}
BRN küçük batch: {'mean': -0.011972633190453053, 'std': 1.002089023590088, 'min': -4.343570709228516, 'max': 4.0954813957214355}
BRN büyük batch: {'mean': 9.5534254796803e-05, 'std': 1.001241683959961, 'min': -4.5844340324401855, 'max': 4.662753105163574}



## 9) İleri seviye pratikler (kısa)

- `warmup_steps` ile renorm’u yavaş açmak genelde iyi fikir.
- Pretrained finetune’da BN freeze bazen daha iyi olabilir.
- Detection/segmentation’da GN popüler; BatchRenorm “BN tabanlı pipeline”ı minimal değiştirmek için iyi seçenek.

### Hızlı kullanım reçetesi
```python
model = convert_bn_to_batchrenorm(model, rmax=3.0, dmax=5.0, warmup_steps=5000)
```
